In [1]:
import os
import numpy as np
import cv2
import matplotlib.pyplot as plt
from skimage.transform import resize
from skimage.feature import hog
from skimage import exposure
from skimage.io import imread, imshow
import pickle
import random
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

import torch
import torch.nn as nn
import torchvision.models as models
import torchvision.transforms as transforms
from torch.autograd import Variable
from PIL import Image

from sklearn.preprocessing import normalize
import timm

2022-06-24 14:24:53.831733: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/ameeth/Wclass/lib/python3.8/site-packages/cv2/../../lib64:/usr/lib/x86_64-linux-gnu/gazebo-11/plugins:/opt/ros/foxy/opt/yaml_cpp_vendor/lib:/opt/ros/foxy/opt/rviz_ogre_vendor/lib:/opt/ros/foxy/lib/x86_64-linux-gnu:/opt/ros/foxy/lib:/usr/local/lib:
2022-06-24 14:24:53.831773: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
#dir='/home/ameeth/WC/MWI'
dir='/home/ameeth/WC/1125/Multi-class Weather Dataset/'

In [3]:
scaler=transforms.Resize(size=(224,224))

In [ ]:
#categories=['HAZE','RAINY','SUNNY','SNOWY']
categories=['Cloudy','Rain','Shine','Sunrise']

model = models.vit_l_32(pretrained=True)
model.eval()
scaler = transforms.Resize(size=(224, 224))
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])
to_tensor = transforms.ToTensor()

Downloading: "https://download.pytorch.org/models/vit_l_32-c7638314.pth" to /home/ameeth/.cache/torch/hub/checkpoints/vit_l_32-c7638314.pth


  0%|          | 0.00/1.14G [00:00<?, ?B/s]

In [14]:
model

VisionTransformer(
  (conv_proj): Conv2d(3, 768, kernel_size=(32, 32), stride=(32, 32))
  (encoder): Encoder(
    (dropout): Dropout(p=0.0, inplace=False)
    (layers): Sequential(
      (encoder_layer_0): EncoderBlock(
        (ln_1): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
        (self_attention): MultiheadAttention(
          (out_proj): NonDynamicallyQuantizableLinear(in_features=768, out_features=768, bias=True)
        )
        (dropout): Dropout(p=0.0, inplace=False)
        (ln_2): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
        (mlp): MLPBlock(
          (linear_1): Linear(in_features=768, out_features=3072, bias=True)
          (act): GELU()
          (dropout_1): Dropout(p=0.0, inplace=False)
          (linear_2): Linear(in_features=3072, out_features=768, bias=True)
          (dropout_2): Dropout(p=0.0, inplace=False)
        )
      )
      (encoder_layer_1): EncoderBlock(
        (ln_1): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
 

In [15]:
model.heads=nn.Identity()

In [16]:
def get_vector(image_name):
    # 1. Load the image with Pillow library
    img = Image.open(image_name) 
    img = img.convert("RGB")  
    #print(img.getbands())
    #img.show()
    # 2. Create a PyTorch Variable with the transformed image
    t_img = Variable(normalize(to_tensor(scaler(img))).unsqueeze(0))    
    # 3. Create a vector of zeros that will hold our feature vector
    #    The 'avgpool' layer has an output size of 512
    my_embedding = torch.zeros(1,2048)    
    # 4. Define a function that will copy the output of a layer
    
    #def copy_data(m, i, o):
        #my_embedding.copy_(o.data)    
    # 5. Attach that function to our selected layer
    #h = layer.register_forward_hook(copy_data)    
    # 6. Run the model on our transformed image
    #model(t_img)    
    # 7. Detach our copy function from the layer
    #h.remove() 
    
    # 8. Return the feature vector
    my_embedding=model(t_img)
    return my_embedding

In [17]:
count=0
data=[]
for category in categories:
    path= os.path.join(dir, category)
    label=categories.index(category)
    
    for image in os.listdir(path):
        image_path=os.path.join(path,image)
        image_vector=get_vector(image_path)
        image_list_vector=image_vector.tolist()
        image_numpy_vector=np.array(image_list_vector,dtype=object)
        #print(image_numpy_vector.shape)
        
        row=image_numpy_vector.shape[0];
        col=image_numpy_vector.shape[1];
        #print(row,col)
        print(count,end=' ')
        count=count+1
        image_new=np.reshape(image_numpy_vector,(row*col));
        image_new1=image_new.astype('float')
        #print(image_new1.shape)
        data.append([image_new1,label])
    

0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197 198 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215 216 217 218 219 220 221 222 223 224 225 226 227 228 229 230 231 232 233 234 235 236 237 238 239 240 241 242 243 244 245 246 247 248 249 250 251 252 253 254 255 256 257 258 259 260 261 262 263 264 265 266 267 268 269 270 271 272 273 274 275 276 27

In [18]:
print(len(data))

1125


In [19]:
pickle.dump(data,open('vit_b_32_1125.pickle','wb'))